<a href="https://colab.research.google.com/github/Dogdriip/notebooks/blob/main/TextRank_newspaper_sentence_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install konlpy

In [2]:
from konlpy.tag import Kkma
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np

In [3]:
kkma = Kkma()
print(kkma.pos("안녕하세요? 오늘 날씨가 참 좋네요."))

[('안녕', 'NNG'), ('하', 'XSV'), ('세요', 'EFN'), ('?', 'SF'), ('오늘', 'NNG'), ('날씨', 'NNG'), ('가', 'JKS'), ('참', 'MAG'), ('좋', 'VA'), ('네요', 'EFN'), ('.', 'SF')]


In [4]:
class SentenceTokenizer(object):
  def __init__(self):
    self.kkma = Kkma()
    self.okt = Okt()
    self.stopwords = ['중인' ,'만큼', '마찬가지', '꼬집었', "연합뉴스", "데일리", "동아일보", "중앙일보", "조선일보", "기자","아", "휴", "아이구", "아이쿠", "아이고", "어", "나", "우리", "저희", "따라", "의해", "을", "를", "에", "의", "가",]

  def text2sentences(self, url):
    sentences = self.kkma.sentences(text)
    for idx in range(len(sentences)):
      if len(sentences[idx]) <= 10:  # if length of current sentence is lte 10, concat next sentence with current sentence.
        sentences[idx - 1] += (" " + sentences[idx])
        sentences[idx] = ""

    return sentences
  
  def get_nouns(self, sentences):
    nouns = []
    for sentence in sentences:
      if sentence is not "":
        nouns.append(" ".join([noun for noun in self.okt.nouns(str(sentence)) if noun not in self.stopwords and len(noun) > 1]))
    
    return nouns


In [5]:
class GraphMatrix(object):
  def __init__(self):
    self.tfidf = TfidfVectorizer()
    self.cnt_vec = CountVectorizer()
    self.graph_sentence = []
  
  # 명사로 이루어진 문장을 입력받아 sklearn의 TfidfVectorizer.fit_transform을 이용하여 tfidf matrix를 만든 후 Sentence graph를 return 한다.
  def build_sent_graph(self, sentence):
    tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
    self.graph_sentence = np.matmul(tfidf_mat, tfidf_mat.T)
    return self.graph_sentence

  # 명사로 이루어진 문장을 입력받아 sklearn의 CountVectorizer.fit_transform을 이용하여 matrix를 만든 후 word graph와 {idx: word}형태의 dictionary를 return한다.
  def build_words_graph(self, sentence):
    cnt_vec_mat = normalize(self.cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
    vocab = self.cnt_vec.vocabulary_
    return np.matmul(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}


In [6]:
class Rank(object):
  def get_ranks(self, graph, d=0.85):
    A = graph
    matrix_size = A.shape[0]
    for id in range(matrix_size):
      A[id, id] = 0  # diagonal goes to zero
      link_sum = np.sum(A[:, id])
      if link_sum != 0:
        A[:, id] /= link_sum
      A[:, id] *= -d
      A[id, id] = 1

    B = (1 - d) * np.ones((matrix_size, 1))
    ranks = np.linalg.solve(A, B)  # 연립방정식 Ax = B
    return {idx: r[0] for idx, r in enumerate(ranks)}


In [7]:
class TextRank(object):
  def __init__(self, text):
    self.sent_tokenize = SentenceTokenizer()  # What we've created
    self.sentences = self.sent_tokenize.text2sentences(text)
    self.nouns = self.sent_tokenize.get_nouns(self.sentences)

    self.graph_matrix = GraphMatrix()  # Also what we've created
    self.sent_graph = self.graph_matrix.build_sent_graph(self.nouns)  # Build sentence graph (adjacency matrix) with given nouns
    self.words_graph, self.idx2word = self.graph_matrix.build_words_graph(self.nouns)  # Build words graph(?) with given nouns

    self.rank = Rank()  # Also what we've created
    self.sent_rank_idx = self.rank.get_ranks(self.sent_graph)
    self.sorted_sent_rank_idx = sorted(self.sent_rank_idx, key=lambda k: self.sent_rank_idx[k], reverse=True)
    self.word_rank_idx = self.rank.get_ranks(self.words_graph)
    self.sorted_word_rank_idx = sorted(self.word_rank_idx, key=lambda k: self.word_rank_idx[k], reverse=True)

  def summarize(self, sent_num=3):
    summary = []
    index = []
    for idx in self.sorted_sent_rank_idx[:sent_num]:
      index.append(idx)
    index.sort()

    for idx in index:
      summary.append(self.sentences[idx])

    return summary

  def keywords(self, word_num=10):
    rank = Rank()
    rank_idx = rank.get_ranks(self.words_graph)
    sorted_rank_idx = sorted(rank_idx, key=lambda k: rank_idx[k], reverse=True)

    keywords = []
    index = []
    for idx in sorted_rank_idx[:word_num]:
      index.append(idx)
    
    for idx in index:
      keywords.append(self.idx2word[idx])

    return keywords
  

In [9]:
text = """
아들의 군 휴가 특혜 의혹과 관련해 검찰로부터 무혐의 처분을 받은 추미애 법무부 장관이 30일 "제보자의 일방적인 주장을 검증하지 않고 단지 정쟁의 도구로 삼은 무책임한 세력들은 반드시 엄중한 책임을 져야 한다"며 경고하고 나섰다. 이에 국민의힘은 "적반하장에 기가 찰 노릇"이라며 반박했다.
추 장관이 이날 오전 8시쯤 페이스북에 국민에게 사과의 뜻을 밝히면서도 야권과 언론을 향해 유감을 표명했다. 검찰의 무혐의 처분 이후 이틀 만에 반격에 나선 것이다.
추 장관은 아들 서모씨(27)의 군 휴가 특혜 관련 의혹에 대해 "정치공세의 성격이 짙은 무리한 고소·고발로 국론을 분열시키고 국력을 소모한 사건"으로 규정하고, "무책임한 세력들은 반드시 엄중한 책임을 져야 할 것이다. 합당한 사과가 없을 시 후속 조치를 취하겠다"고 경고했다.
언론을 향해서도 "보도 양태에 깊은 유감을 보내지 않을 수 없다"며 "사실과 진실을 짚는 대신 허위의 주장을 그대로 싣고 나아가 허위를 사실인 양 보도한 다수 언론은 국민에게 커다란 실망과 상처를 줬다"고 말했다.
'무책임한 세력'이 누구인지는 직접 밝히진 않았지만 국민의힘 등 야권과 보수 언론을 겨냥한 것으로 풀이된다.
이에 장제원 국민의힘 의원은 자신의 페이스북에 "'방귀 낀 X이 성낸다'라는 말이 있다. 추 장관의 적반하장에 기가 찰 노릇"이라고 밝혔다.
장 의원은 이어 "추 장관이 수사 관련 자료가 공개돼 자신의 거짓말이 탄로가 나자, 사과는 커녕 국민과 언론을 향해 겁박까지 하고 나섰다"고 지적했다.
장 의원은 또 "'반드시 엄중한 책임을 져야 한다' 라구요"라고 반문한 뒤 "국민 앞에서 눈 하나 깜짝하지 않고 했던 거짓말부터 엄중한 책임을 져야 할 것"이라고 반박했다.
그러면서 "추 장관은 '합당한 사과가 없을 시 후속 조치를 취할 것' 이라며 협박도 서슴지 않는다"며 "저희들이 하고 싶은 말이고, 추 장관이 했던 거짓말에 대해 합당한 사과가 없을 시, 국민과 함께 후속조치를 취해 나갈 것"이라고 경고했다.
국민의힘 서울 송파병 당협위원장을 맡고 있는 김근식 경남대 교수도 이날 페이스북에 "추 장관님이 국민에게 거짓말한 것부터 사과해야 한다"며 "먼저 죄없는 젊은이를 거짓말쟁이로 낙인찍은 것을 사과해야 한다"고 지적했다.
또 추 장관이 '검찰개혁 완수'를 언급한 것과 관련해선 "제발 이제는 검찰개혁이란 말 좀 그만하라"며 "국민들은 이제 검찰개혁이라 쓰고 '검찰 길들이기'라고 읽는다"고 덧붙였다.
안혜진 국민의당 논평에서 "부정과 부조리, 비상식적인 짓을 해도 내 편이기만 하면 무조건 보호받는 나라가 대통령께서 꿈꾸었던 나라는 아닐 것"이라고 말했다.
"""

textrank = TextRank(text)
for row in textrank.summarize():
  print(row)
  print()


'\n아들의 군 휴가 특혜 의혹과 관련해 검찰로부터 무혐의 처분을 받은 추미애 법무부 장관이 30일 "제보자의 일방적인 주장을 검증하지 않고 단지 정쟁의 도구로 삼은 무책임한 세력들은 반드시 엄중한 책임을 져야 한다"며 경고하고 나섰다. 이에 국민의힘은 "적반하장에 기가 찰 노릇"이라며 반박했다.\n추 장관이 이날 오전 8시쯤 페이스북에 국민에게 사과의 뜻을 밝히면서도 야권과 언론을 향해 유감을 표명했다. 검찰의 무혐의 처분 이후 이틀 만에 반격에 나선 것이다.\n추 장관은 아들 서모씨(27)의 군 휴가 특혜 관련 의혹에 대해 "정치공세의 성격이 짙은 무리한 고소·고발로 국론을 분열시키고 국력을 소모한 사건"으로 규정하고, "무책임한 세력들은 반드시 엄중한 책임을 져야 할 것이다. 합당한 사과가 없을 시 후속 조치를 취하겠다"고 경고했다.\n언론을 향해서도 "보도 양태에 깊은 유감을 보내지 않을 수 없다"며 "사실과 진실을 짚는 대신 허위의 주장을 그대로 싣고 나아가 허위를 사실인 양 보도한 다수 언론은 국민에게 커다란 실망과 상처를 줬다"고 말했다.\n\'무책임한 세력\'이 누구인지는 직접 밝히진 않았지만 국민의힘 등 야권과 보수 언론을 겨냥한 것으로 풀이된다.\n이에 장제원 국민의힘 의원은 자신의 페이스북에 "\'방귀 낀 X이 성낸다\'라는 말이 있다. 추 장관의 적반하장에 기가 찰 노릇"이라고 밝혔다.\n장 의원은 이어 "추 장관이 수사 관련 자료가 공개돼 자신의 거짓말이 탄로가 나자, 사과는 커녕 국민과 언론을 향해 겁박까지 하고 나섰다"고 지적했다.\n장 의원은 또 "\'반드시 엄중한 책임을 져야 한다\' 라구요"라고 반문한 뒤 "국민 앞에서 눈 하나 깜짝하지 않고 했던 거짓말부터 엄중한 책임을 져야 할 것"이라고 반박했다.\n그러면서 "추 장관은 \'합당한 사과가 없을 시 후속 조치를 취할 것\' 이라며 협박도 서슴지 않는다"며 "저희들이 하고 싶은 말이고, 추 장관이 했던 거짓말에 대해 합당한 사과가 없을 시, 국민과 함께 후속조치를 취해 나갈 것

Config option `delete_to_trash` not recognized by `ColabFileContentsManager`.
Writing notebook server cookie secret to /root/.local/share/jupyter/runtime/notebook_cookie_secret
google.colab serverextension initialized.
Serving notebooks from local directory: /
0 active kernels
The Jupyter Notebook is running at:
http://172.28.0.2:9000/
Use Control-C to stop this server and shut down all kernels (twice to skip confirmation).
Kernel started: 547409b1-5332-4bfc-b312-241e03cbe9bc
Adapting to protocol v5.1 for kernel 547409b1-5332-4bfc-b312-241e03cbe9bc
KernelRestarter: restarting kernel (1/5), keep random ports
KernelRestarter: restarting kernel (1/5), keep random ports
Adapting to protocol v5.1 for kernel 547409b1-5332-4bfc-b312-241e03cbe9bc
Adapting to protocol v5.1 for kernel 547409b1-5332-4bfc-b312-241e03cbe9bc
Adapting to protocol v5.1 for kernel 547409b1-5332-4bfc-b312-241e03cbe9bc
Adapting to protocol v5.1 for kernel 547409b1-5332-4bfc-b312-241e03cbe9bc
KernelRestarter: restarting k